In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [2]:
import time
from typing import List, Set
from urllib.parse import quote_plus

import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    )
}


def fetch_job_ids(
    title: str,
    location: str,
    num_pages: int,
    pause: float = 0.5,
    verbose: bool = True,
) -> List[str]:
    """
    Busca job_ids de vagas no LinkedIn Jobs (endpoint público 'seeMoreJobPostings').

    :param title: termo de busca (ex: 'engineer', 'scientist').
    :param location: localização (ex: 'Brazil', 'Lisbon, Portugal').
    :param num_pages: número de páginas a percorrer.
    :param pause: tempo de espera entre requisições (segundos).
    :param verbose: se True, imprime logs simples.
    :return: lista de job_ids (sem duplicados).
    """
    job_ids: Set[str] = set()

    # Prepara parâmetros com encoding seguro para URL
    keywords = f"data {title}".strip()
    keywords_encoded = quote_plus(keywords)
    location_encoded = quote_plus(location)

    with requests.Session() as session:
        session.headers.update(HEADERS)

        for page in range(num_pages):
            start = page * 10  # cada página costuma ter 10 vagas
            list_url = (
                "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
                f"?keywords={keywords_encoded}"
                f"&location={location_encoded}"
                "&position=1&pageNum=0"
                f"&start={start}"
            )

            try:
                response = session.get(list_url, timeout=10)
            except requests.RequestException as e:
                if verbose:
                    print(f"[ERRO] Falha na requisição da página {page}: {e}")
                continue

            if verbose:
                print(f"[INFO] Página {page} - status {response.status_code}")

            if response.status_code != 200:
                # 429 = too many requests, 404, etc.
                if verbose:
                    print(f"[AVISO] Pulando página {page} (status {response.status_code})")
                continue

            soup = BeautifulSoup(response.text, "html.parser")
            page_jobs = soup.find_all("li")

            if not page_jobs and verbose:
                print(f"[AVISO] Nenhuma vaga encontrada na página {page}")

            for job in page_jobs:
                base_card_div = job.find("div", {"class": "base-card"})
                if not base_card_div:
                    continue

                urn = base_card_div.get("data-entity-urn")
                if not urn:
                    continue

                parts = urn.split(":")
                if len(parts) < 4:
                    continue

                job_id = parts[3]
                job_ids.add(job_id)
                if verbose:
                    print(f"[JOB ID] {job_id}")

            # Pequena pausa para evitar 429
            time.sleep(pause)

    return list(job_ids)


In [3]:
import time
from typing import List, Dict, Optional

import requests
from bs4 import BeautifulSoup


HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    )
}


def get_text_or_none(elem) -> Optional[str]:
    """Retorna o texto stripado ou None se elem for None."""
    return elem.get_text(strip=True) if elem else None


def safe_criteria(criteria_list, index: int) -> Optional[str]:
    """Retorna o critério se existir; caso contrário, None."""
    try:
        return criteria_list[index].get_text(strip=True)
    except (IndexError, AttributeError):
        return None


def parse_job_page(html: str, job_id: str) -> Dict[str, Optional[str]]:
    """Extrai os dados de uma página de vaga do LinkedIn."""
    soup = BeautifulSoup(html, "html.parser")
    job_post: Dict[str, Optional[str]] = {}

    # Título da vaga
    job_post["job_title"] = get_text_or_none(
        soup.find(
            "h2",
            {
                "class": (
                    "top-card-layout__title font-sans text-lg "
                    "papabear:text-xl font-bold leading-open "
                    "text-color-text mb-0 topcard__title"
                )
            },
        )
    )

    # Nome da empresa
    job_post["company_name"] = get_text_or_none(
        soup.find(
            "a",
            {"class": "topcard__org-name-link topcard__flavor--black-link"},
        )
    )

    # Critérios (nível de experiência, tipo de contrato, etc.)
    criteria = soup.find_all(
        "span",
        {
            "class": (
                "description__job-criteria-text "
                "description__job-criteria-text--criteria"
            )
        },
    )

    job_post["experience_level"] = safe_criteria(criteria, 0)
    job_post["type_of_contract"] = safe_criteria(criteria, 1)
    # Se quiser voltar com os outros, é só descomentar:
    # job_post["job_role"] = safe_criteria(criteria, 2)
    # job_post["field"] = safe_criteria(criteria, 3)

    # Easy apply (True/False)
    try:
        offsite_icon = soup.find(
            "icon",
            {"data-svg-class-name": "apply-button__offsite-apply-icon-svg"},
        )
        # Se tem ícone de offsite, não é easy apply
        job_post["easy_apply"] = False if offsite_icon else True
    except Exception:
        job_post["easy_apply"] = None

    # Tempo desde a postagem
    job_post["time_posted"] = get_text_or_none(
        soup.find(
            "span",
            {
                "class": (
                    "posted-time-ago__text "
                    "topcard__flavor--metadata"
                )
            },
        )
    )

    # Número de candidatos
    job_post["num_applicants"] = get_text_or_none(
        soup.find("span", {"class": "num-applicants__caption"})
    )

    # Link da vaga
    job_post["job_link"] = f"https://www.linkedin.com/jobs/view/{job_id}/"

    return job_post


def scrape_jobs(id_list: List[str], sleep_seconds: float = 0.5) -> List[Dict]:
    """Faz scrape de uma lista de job_ids do LinkedIn."""
    job_list: List[Dict] = []

    with requests.Session() as session:
        session.headers.update(HEADERS)

        for job_id in id_list:
            job_url = (
                f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
            )

            try:
                response = session.get(job_url, timeout=10)
            except requests.RequestException as e:
                print(f"Erro ao requisitar {job_url}: {e}")
                continue

            print(job_id, response.status_code)

            # 200 = ok; 429 = many requests; 404 = não encontrado etc.
            if response.status_code != 200:
                # aqui você pode decidir logar / armazenar o erro
                continue

            job_post = parse_job_page(response.text, job_id)
            job_list.append(job_post)

            # Uma pequena pausa ajuda a evitar 429
            time.sleep(sleep_seconds)

    return job_list


In [4]:
title = "engineer"
location = "Brazil"
num_pages = 5

id_list = fetch_job_ids(title, location, num_pages)
##print("Total de IDs coletados:", len(id_list))


[INFO] Página 0 - status 200
[JOB ID] 4318327757
[JOB ID] 4308497997
[JOB ID] 4281828542
[JOB ID] 4233957493
[JOB ID] 4337472760
[JOB ID] 4312919215
[JOB ID] 4298239221
[JOB ID] 4311720804
[JOB ID] 4324753186
[JOB ID] 4310476364
[INFO] Página 1 - status 200
[JOB ID] 4317491191
[JOB ID] 4303420983
[JOB ID] 4231033954
[JOB ID] 4312301348
[JOB ID] 4316823422
[JOB ID] 4331368566
[JOB ID] 4314223388
[JOB ID] 3969555543
[JOB ID] 4334041808
[JOB ID] 4315447335
[INFO] Página 2 - status 200
[JOB ID] 4331348122
[JOB ID] 4334386540
[JOB ID] 4307252914
[JOB ID] 4245979421
[JOB ID] 4333019146
[JOB ID] 4320660148
[JOB ID] 4307685458
[JOB ID] 4214699583
[JOB ID] 4295464589
[JOB ID] 4313394952
[INFO] Página 3 - status 200
[JOB ID] 4312861310
[JOB ID] 4334181171
[JOB ID] 4318515516
[JOB ID] 4320613907
[JOB ID] 4091160916
[JOB ID] 4337456861
[JOB ID] 4333210689
[JOB ID] 4304754197
[JOB ID] 4318523103
[JOB ID] 4305932314
[INFO] Página 4 - status 200
[JOB ID] 4307544126
[JOB ID] 4320251804
[JOB ID] 430478

In [5]:
id_list = ["1234567890", "0987654321"]
jobs = scrape_jobs(id_list)


1234567890 404
0987654321 404
